In [1]:
out_path = "../../corpus/novel_formated/"
corpus_name = "novel_segments2.tsv"

In [2]:
import os
import sys
import numpy as np
import pandas as pd

In [3]:
import csv
conv_data = []
with open(out_path+corpus_name, "r") as f:
    reader = csv.reader(f, delimiter="\t")
    conv_data = [row for row in reader]

In [4]:
conv_only_utt = [ ["".join(conv[-3:-1]), conv[-1]] for conv in conv_data]

In [5]:
# トリプルの作成
# アンカー，ポジティブ，ネガティブ
import random
random.seed(0)
def make_triple(conv_data):
    triple_list = []
    last_utts = [ conv[-1] for conv in conv_data if len(conv[-1]) > 6 ]
    neg_candidate = random.sample(last_utts, len(last_utts))
    for conv, neg in zip(conv_data, neg_candidate):
        # 短すぎる応答は殺す
        if len(conv[1]) <= 6:
            continue
        triple_list.append( [*conv , neg] )

    return triple_list



In [6]:
triple_list = make_triple(conv_only_utt)

In [7]:
len(triple_list)

715320

In [16]:
mini_size = 3000
triple_list_mini = random.sample(triple_list,  mini_size)

In [17]:
lasts = [ conv[-1] for conv in conv_only_utt if len(conv[-1]) > 6 ]

In [18]:
def to_tsv(filename, triplet):
    with open(filename, "w") as f:
        lines = ["{0}\t{1}\t{2}".format(*tri) for tri in triplet]
        f.write("\n".join(lines)+"\n")

In [19]:
# 学習用，検証用，テスト用の分割
from sklearn.model_selection import train_test_split
train, dev_test = train_test_split(triple_list_mini, train_size=0.8, random_state=4)
dev, test = train_test_split(dev_test, train_size=0.5, random_state=7)

In [20]:
data_path = "../../corpus/coherence_sbert_data/"

In [21]:
len(train)

2400

In [22]:
to_tsv(data_path+"triplet_train.tsv", train)
to_tsv(data_path+"triplet_dev.tsv", dev)
to_tsv(data_path+"triplet_test.tsv", test)

In [23]:
BATCH_SIZE = 16
NUM_EPOCHS = 1
EVAL_STEPS = 1000
WARMUP_STEPS = int(len(triple_list_mini) // BATCH_SIZE * 0.1) 